# Proyecto II: Shakespear-Matic

Debido a la falta de producción literaria de calidad (subjetivamente medida por el profesor Alexander), el profesor  Alexander ha decidio explorar la generación automatica de texto, para ver si de esa forma se puede lograr mejores obras contemporaneas. Debido a que los estudiantes del curso de NLP (Natural Language Processing) ya son expertos en la generación automatica de texto, el profesor ha decidio generar el siguiente reto para evaluar sus conocimientos y la calidad de estas herramientas  en la producción literaria.

Para esta tarea el profesor quiere que los estudiantes entrenen diferentes sistemas para la generación de tres estilos literarios diferentes, ente los estilos que se desean observar los estudiantes pueden escoger entre: Novelas clásicas, cuentos de niños, poesia, letras de canciones (por favor NO regueton), obras de realismo mágico, entre otras.

Para el desarrollo de esta tarea el profesor Alexander pide que se diseñen sistemas de generación automatica basada en palabras, para lo cual deben implementar en el pipeline:

1. Adquisición de datos.
2. Generación de vectors de palabras usando word2vec.
3. Entrenamiento de los sistemas para los estilos literarios escogidos.
4. Predicción del texto:
    * Si la codificación se hizo con onehot encoding para la salida, el sistema produce de salida la siguiente palabra a generar.
    * Si la codificación de salida se hizo con word2vec se debe implementar una busqueda de la palbra más cercana a la codificación generada por la salida de la red.
      
Tengan en cuenta que la relación entre la longitud del corpus y las palabras del vocabulario debe ser adecuada, para poder tener buenos resultados. El texto generado debe tener tambien en cuenta la puntuación.

AL finalizar el proyecto, el profesor Alexander quiere que los estudiantes contesten las siguientes preguntas:

1. ¿Qué pueden observar de los resusltados de los sistemas de generacion automatica para los tres estilos literarios diferentes? ¿Porqué considera que los resultados son de está forma?

2. ¿Qué tamaño de N escogierón para la codificación de las palabras en vectores? ¿?Qué sucede si escogen un valor diferente de N?

3. ¿Qué sucede si se cambia el número de palbras anteriores utilizados para predecir la palabra siguiiente en el modelo?

4. ¿Qué ventajas y desventajas tienen las diferentes formas de codificar la salida - one hot encoding y word2vec?

5. ¿Qué forma de codificación de la salida escogierón al final? ¿Porqué?

6. ¿Qué pueden concluir del proyecto? ¿Cómo se pueden mejorar los resultados?

Por favor al desarrollar el proyecto comentar cada bloque de codigo con un analisis de lo que esperan lograr y un análisis de los resultados preliminares. El proyecto se debe entregar a más tardar el **Domingo 2 de Mayo del 2021 a las 11:59 p.m. hora Bogotá**.

In [3]:
import re 
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [8]:
# Función para leer y procesar el archivo en texto plano.

def get_file_data(fname, stop_word_removal='no'):
    file_contents = []
    with open(fname,encoding="utf-8") as f:
        file_contents = f.read()
    text = []
    for val in file_contents.split('.'):
        val = re.sub(r'[,¡!¿?;-]+','.',val)
        val = re.sub(r'á','a',val)
        val = re.sub(r'é','e',val)
        val = re.sub(r'í','i',val)
        val = re.sub(r'ó','o',val)
        val = re.sub(r'ú','u',val)
        val = re.sub(r'Á','A',val)
        val = re.sub(r'É','E',val)
        val = re.sub(r'Í','I',val)
        val = re.sub(r'Ó','O',val)
        val = re.sub(r'Ú','U',val)
        val = re.sub(r'ñ','n',val)
        val = re.sub(r'Ñ','N',val)
        sent = re.findall("[A-Za-z]+", val)
        line = ''
        for words in sent:
            
            if stop_word_removal == 'yes': 
                if len(words) > 1 and words not in stop_words:
                    line = line + ' ' + words
            else:
                if len(words) > 1 :
                    line = line + ' ' + words
        text.append(line)
    return text

# Función para obtener un Vocabulario en función del texto procesado

def generate_dictinoary_data(text):
    word_to_index= dict()
    index_to_word = dict()
    corpus = []
    count = 0
    vocab_size = 0
    
    for row in text:
        for word in row.split():
            word = word.lower()
            corpus.append(word)
            if word_to_index.get(word) == None:
                word_to_index.update ( {word : count})
                index_to_word.update ( {count : word })
                count  += 1
    vocab_size = len(word_to_index)
    length_of_corpus = len(corpus)
    
    return word_to_index,index_to_word,corpus,vocab_size,length_of_corpus

# Función para generar representaciones one hot de los vectores target y del corpus

def get_one_hot_vectors(target_word,context_words,vocab_size,word_to_index):
    
    #Create an array of size = vocab_size filled with zeros
    trgt_word_vector = np.zeros(vocab_size)
    
    #Get the index of the target_word according to the dictionary word_to_index. 
    index_of_word_dictionary = word_to_index.get(target_word) 
    
    #Set the index to 1
    trgt_word_vector[index_of_word_dictionary] = 1
    
    #Repeat same steps for context_words but in a loop
    ctxt_word_vector = np.zeros(vocab_size)
    
    
    for word in context_words:
        index_of_word_dictionary = word_to_index.get(word) 
        ctxt_word_vector[index_of_word_dictionary] = 1
        
    return trgt_word_vector,ctxt_word_vector

# Función para generar los datos de entrenamiento para la red neuronal que representa el modelo word2vec

def generate_training_data(corpus,window_size,vocab_size,word_to_index,length_of_corpus,sample=None):

    training_data =  []
    training_sample_words =  []
    for i,word in enumerate(corpus):

        index_target_word = i
        target_word = word
        context_words = []

        #when target word is the first word
        if i == 0:  

            # trgt_word_index:(0), ctxt_word_index:(1,2)
            context_words = [corpus[x] for x in range(i + 1 , window_size + 1)] 


        #when target word is the last word
        elif i == len(corpus)-1:

            # trgt_word_index:(9), ctxt_word_index:(8,7), length_of_corpus = 10
            context_words = [corpus[x] for x in range(length_of_corpus - 2 ,length_of_corpus -2 - window_size  , -1 )]

        #When target word is the middle word
        else:

            #Before the middle target word
            before_target_word_index = index_target_word - 1
            for x in range(before_target_word_index, before_target_word_index - window_size , -1):
                if x >=0:
                    context_words.extend([corpus[x]])

            #After the middle target word
            after_target_word_index = index_target_word + 1
            for x in range(after_target_word_index, after_target_word_index + window_size):
                if x < len(corpus):
                    context_words.extend([corpus[x]])


        trgt_word_vector,ctxt_word_vector = get_one_hot_vectors(target_word,context_words,vocab_size,word_to_index)
        training_data.append([trgt_word_vector,ctxt_word_vector])   
        
        if sample is not None:
            training_sample_words.append([target_word,context_words])   
        
    return training_data,training_sample_words

In [9]:
fname = 'cuento.txt' #Nombre del archivo
C = 2 # Número de palabras de contexto a la derecha y a la izquierda
text = get_file_data(fname, stop_word_removal='no')
word_to_index,index_to_word,corpus,vocab_size,length_of_corpus = generate_dictinoary_data(text)
training_data,training_sample_words = generate_training_data(corpus,C,vocab_size,word_to_index,length_of_corpus,sample=None)

In [10]:
# Matriz de entrada y de salida
X = np.array([x[1] for x in training_data])
Y = np.array([y[0] for y in training_data])

In [11]:
# Definiendo red neuronal
model = Sequential()  
model.add(Dense(100,input_shape=(vocab_size,), activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='CategoricalCrossentropy', optimizer='rmsprop', metrics=['accuracy'])  

In [13]:
# Entrenando
model.fit(X,Y,epochs=100)

Epoch 1/100
387/387 [==============================] - 4s 10ms/step - loss: 2.4177 - accuracy: 0.6263
Epoch 2/100
387/387 [==============================] - 4s 10ms/step - loss: 2.3757 - accuracy: 0.6326
Epoch 3/100
387/387 [==============================] - 4s 11ms/step - loss: 2.3337 - accuracy: 0.6376
Epoch 4/100
387/387 [==============================] - 4s 11ms/step - loss: 2.2903 - accuracy: 0.6415
Epoch 5/100
387/387 [==============================] - 4s 11ms/step - loss: 2.2489 - accuracy: 0.6448
Epoch 6/100
387/387 [==============================] - 4s 11ms/step - loss: 2.2082 - accuracy: 0.6511
Epoch 7/100
387/387 [==============================] - 4s 10ms/step - loss: 2.1667 - accuracy: 0.6560
Epoch 8/100
387/387 [==============================] - 4s 10ms/step - loss: 2.1262 - accuracy: 0.6602
Epoch 9/100
387/387 [==============================] - 5s 12ms/step - loss: 2.0868 - accuracy: 0.6653
Epoch 10/100
387/387 [==============================] - 4s 9ms/step - loss: 2.0473

387/387 [==============================] - 2s 6ms/step - loss: 0.3005 - accuracy: 0.9420
Epoch 82/100
387/387 [==============================] - 3s 7ms/step - loss: 0.2925 - accuracy: 0.9440
Epoch 83/100
387/387 [==============================] - 3s 7ms/step - loss: 0.2843 - accuracy: 0.9444
Epoch 84/100
387/387 [==============================] - 3s 7ms/step - loss: 0.2748 - accuracy: 0.9464
Epoch 85/100
387/387 [==============================] - 3s 7ms/step - loss: 0.2663 - accuracy: 0.9475
Epoch 86/100
387/387 [==============================] - 3s 7ms/step - loss: 0.2586 - accuracy: 0.9492
Epoch 87/100
387/387 [==============================] - 3s 7ms/step - loss: 0.2506 - accuracy: 0.9503
Epoch 88/100
387/387 [==============================] - 3s 7ms/step - loss: 0.2432 - accuracy: 0.9520
Epoch 89/100
387/387 [==============================] - 3s 7ms/step - loss: 0.2367 - accuracy: 0.9530
Epoch 90/100
387/387 [==============================] - 3s 7ms/step - loss: 0.2294 - accuracy: 

In [20]:
# Obteniendo representaciones de los vectores con las matrices de pesos
W1 = model.get_weights()[0]
W2 = model.get_weights()[2]
W_c = 0.5*(W2.T+W1)

In [ ]:
def getIndex(D,v):
    k = np.where(D==v)[0].tolist()
    i = max(set(k),key=k.count)
    return i


In [16]:
fname = 'novela.txt' #Nombre del archivo
C = 2 # Número de palabras de contexto a la derecha y a la izquierda
text = get_file_data(fname, stop_word_removal='no')
word_to_index,index_to_word,corpus,vocab_size,length_of_corpus = generate_dictinoary_data(text)
training_data,training_sample_words = generate_training_data(corpus,C,vocab_size,word_to_index,length_of_corpus,sample=None)

In [17]:
X = np.array([x[1] for x in training_data])
Y = np.array([y[0] for y in training_data])

In [18]:
# Definiendo red neuronal
model = Sequential()  
model.add(Dense(100,input_shape=(vocab_size,), activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='CategoricalCrossentropy', optimizer='rmsprop', metrics=['accuracy'])  

In [19]:
# Entrenando
model.fit(X,Y,epochs=100)

Epoch 1/100
614/614 [==============================] - 17s 27ms/step - loss: 6.5982 - accuracy: 0.0569
Epoch 2/100
614/614 [==============================] - 13s 21ms/step - loss: 6.1162 - accuracy: 0.0773
Epoch 3/100
614/614 [==============================] - 12s 20ms/step - loss: 5.9366 - accuracy: 0.0862
Epoch 4/100
614/614 [==============================] - 11s 18ms/step - loss: 5.8088 - accuracy: 0.0965
Epoch 5/100
614/614 [==============================] - 10s 16ms/step - loss: 5.7089 - accuracy: 0.1060
Epoch 6/100
614/614 [==============================] - 10s 16ms/step - loss: 5.6294 - accuracy: 0.1115
Epoch 7/100
614/614 [==============================] - 9s 15ms/step - loss: 5.5612 - accuracy: 0.1182
Epoch 8/100
614/614 [==============================] - 9s 14ms/step - loss: 5.5070 - accuracy: 0.1242
Epoch 9/100
614/614 [==============================] - 9s 14ms/step - loss: 5.4622 - accuracy: 0.1326
Epoch 10/100
614/614 [==============================] - 9s 14ms/step - loss:

614/614 [==============================] - 9s 14ms/step - loss: 3.8218 - accuracy: 0.4968
Epoch 81/100
614/614 [==============================] - 8s 13ms/step - loss: 3.8027 - accuracy: 0.4992
Epoch 82/100
614/614 [==============================] - 9s 15ms/step - loss: 3.7820 - accuracy: 0.5010
Epoch 83/100
614/614 [==============================] - 9s 14ms/step - loss: 3.7622 - accuracy: 0.5032
Epoch 84/100
614/614 [==============================] - 9s 14ms/step - loss: 3.7430 - accuracy: 0.5044
Epoch 85/100
614/614 [==============================] - 9s 14ms/step - loss: 3.7208 - accuracy: 0.5063
Epoch 86/100
614/614 [==============================] - 9s 15ms/step - loss: 3.7013 - accuracy: 0.5081
Epoch 87/100
614/614 [==============================] - 9s 15ms/step - loss: 3.6813 - accuracy: 0.5098
Epoch 88/100
614/614 [==============================] - 9s 15ms/step - loss: 3.6623 - accuracy: 0.5116
Epoch 89/100
614/614 [==============================] - 9s 15ms/step - loss: 3.6440 - 